In [1]:
import datetime as datetime
from datetime import datetime, timedelta
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import random
import calendar
from casadi import *
import casadi as cd
from sklearn.linear_model import LinearRegression

In [2]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)

    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)

            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)

            # Append the dataframe to the list
            df_list.append(df)


C:\Users\jaina\AppData\Local\Temp\ipykernel_2704\3450862427.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_2704\3450862427.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_2704\3450862427.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=

In [3]:
bill_path = 'D:\\mlinternship\\IITGuwahatiElectricityBills'
for filename in os.listdir(bill_path):
    # Check if the file has the '.xlsx' extension
    if filename.endswith('.xlsx'):
        file_path = os.path.join(bill_path, filename)
        # Use the custom date parser function
        bill_df = pd.read_excel(file_path)

bill_df['Month'] = pd.to_datetime(bill_df['Month'])
#I assumed its in kilowatts
bill_df['MW'] = bill_df['Number of units of electricity consumed']/1000
bill_df.drop(['Number of units of electricity consumed'], axis=1)
bill_df

,Month,MW
0,2022-01-01,1415.4246
1,2022-02-01,1353.4260
2,2022-03-01,2254.4145
3,2022-04-01,2269.1010
4,2022-05-01,2690.4150
5,2022-06-01,2708.4910
6,2022-07-01,3113.6625
7,2022-08-01,3606.0495
8,2022-09-01,3335.3190
9,2022-10-01,2787.5835


,Month,Number of units of electricity consumed,MW
0,2022-01-01,1415424.6,1415.4246
1,2022-02-01,1353426.0,1353.4260
2,2022-03-01,2254414.5,2254.4145
3,2022-04-01,2269101.0,2269.1010
4,2022-05-01,2690415.0,2690.4150
5,2022-06-01,2708491.0,2708.4910
6,2022-07-01,3113662.5,3113.6625
7,2022-08-01,3606049.5,3606.0495
8,2022-09-01,3335319.0,3335.3190
9,2022-10-01,2787583.5,2787.5835


In [4]:
power_df = pd.concat(df_list, ignore_index=True)
power_df.to_csv('power_datacsv.csv')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
power_df['MW'] = power_df['MW'].replace('nr', np.nan)
power_df = power_df[['Time', 'MW']]
#power_df.dropna(subset=['Time'], inplace=True)
power_df['MW'] = power_df['MW'].astype(str)
power_df['MW'] = pd.to_numeric(power_df['MW'].str.replace(',', '.'), errors='coerce')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df = power_df.sort_values('Time')
power_df.reset_index(drop = True, inplace = True)
power_df

,Time,MW
0,2022-01-16 02:00:00,NaN
1,2022-01-17 02:00:00,NaN
2,2022-01-18 02:00:00,NaN
3,2022-01-19 02:00:00,NaN
4,2022-01-20 02:00:00,NaN
...,...,...
8542,NaT,NaN
8543,NaT,NaN
8544,NaT,NaN
8545,NaT,NaN


In [5]:
# read the temperature data csv
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])
temperature_df['Time'] = pd.DatetimeIndex(temperature_df['Time']) + timedelta(hours=5,minutes=30)
temperature_df['temperature'] = pd.to_numeric(temperature_df['temperature'], errors='coerce')
temperature_df.set_index('Time', inplace=True)
temperature_df['temperature'] = temperature_df['temperature'].interpolate(method='polynomial', order = 5)
temperature_df.reset_index(inplace=True)


In [6]:
df = pd.merge(power_df, temperature_df, on='Time', how='left')
df['temperature'] = df['temperature'].interpolate(method='polynomial', order = 5)
df['Time'] = pd.to_datetime(df['Time'])
full_model_start_time = pd.Timestamp('2023-05-01 00:00:00')
full_model_end_time = pd.Timestamp('2023-06-15 23:00:00')
df = df[(df['Time'] >= full_model_start_time) & (df['Time'] <= full_model_end_time)]
df = df.drop(df[df['MW'] > 20].index)
df = df.sort_values('Time')
df.reset_index(drop=True)
df

,Time,MW,temperature
0,2023-05-01 00:00:00,NaN,27.000000
1,2023-05-01 01:00:00,NaN,33.394639
2,2023-05-01 02:00:00,NaN,33.803576
3,2023-05-01 03:00:00,NaN,30.046507
4,2023-05-01 04:00:00,NaN,27.000000
...,...,...,...
1100,2023-06-15 19:00:00,4.0,25.000000
1101,2023-06-15 20:00:00,4.0,25.000000
1102,2023-06-15 21:00:00,4.0,25.000000
1103,2023-06-15 22:00:00,NaN,25.000000


,Time,MW,temperature
7127,2023-05-01 00:00:00,NaN,27.000000
7128,2023-05-01 01:00:00,NaN,33.394639
7129,2023-05-01 02:00:00,NaN,33.803576
7130,2023-05-01 03:00:00,NaN,30.046507
7131,2023-05-01 04:00:00,NaN,27.000000
...,...,...,...
8227,2023-06-15 19:00:00,4.0,25.000000
8228,2023-06-15 20:00:00,4.0,25.000000
8229,2023-06-15 21:00:00,4.0,25.000000
8230,2023-06-15 22:00:00,NaN,25.000000


In [7]:
'''start_date = datetime(2022, 8, 1, 10, 0, 0)  # August 1st, 2022, 10:00:00 AM
end_date = datetime(2022, 8, 28, 22, 0, 0)  # August 28th, 2022, 10:00:00 PM

# Generate 100 random timestamps within the constraints
random_times = []
for _ in range(100):
    random_hour = random.randint(10, 22)  # Random hour between 10 AM and 10 PM
    minute = 0
    second = 0
    random_time = datetime(2022, 8, random.randint(1, 28), random_hour, minute, second)
    random_times.append(random_time)

# Convert the random timestamps to string format
random_time_strings = [time.strftime('%Y-%m-%d %H:%M:%S') for time in random_times]

# Drop rows with the generated random timestamps from df
df = df[~df['Time'].isin(random_time_strings)]
df'''

"start_date = datetime(2022, 8, 1, 10, 0, 0)  # August 1st, 2022, 10:00:00 AM\nend_date = datetime(2022, 8, 28, 22, 0, 0)  # August 28th, 2022, 10:00:00 PM\n\n# Generate 100 random timestamps within the constraints\nrandom_times = []\nfor _ in range(100):\n    random_hour = random.randint(10, 22)  # Random hour between 10 AM and 10 PM\n    minute = 0\n    second = 0\n    random_time = datetime(2022, 8, random.randint(1, 28), random_hour, minute, second)\n    random_times.append(random_time)\n\n# Convert the random timestamps to string format\nrandom_time_strings = [time.strftime('%Y-%m-%d %H:%M:%S') for time in random_times]\n\n# Drop rows with the generated random timestamps from df\ndf = df[~df['Time'].isin(random_time_strings)]\ndf"

In [8]:
TcoolStPt = 31
CDH = df['temperature'] - TcoolStPt
CDH.clip(lower=0, inplace=True)
CDH = pd.DataFrame(data=CDH.values, columns=['CDH'], index=df.index)
# Concatenate CDH with the original DataFrame using the index
df = pd.concat([df, CDH], axis=1)
df = df.sort_values('Time')
df.reset_index(inplace=True, drop = True)

In [9]:
numOmegas = 24 * 7
num_of_rows = df.shape[0]
omegas = np.zeros((num_of_rows, numOmegas))  # Assuming numOmegas columns for omegas
concatenated_data = np.concatenate((df, omegas), axis=1)
column_names = ['Time', 'MW', 'temperature', 'CDH']
for i in range(1, numOmegas + 1,1):
    column_names.append('omega' + str(i))

df = pd.DataFrame(concatenated_data, columns=column_names)
df['Time'] = pd.to_datetime(df['Time'])
for i in range(0,num_of_rows):
        datetime = df.Time.loc[i]
        hourOfWeekIndex = int(datetime.dayofweek*24+(datetime.hour+1))
        x = np.zeros((1,numOmegas))
        x[0,hourOfWeekIndex-1]=1
        omegas[i,:]=x

df.iloc[:,4:]=omegas
df

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,2023-05-01 00:00:00,NaN,27.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-05-01 01:00:00,NaN,33.394639,2.394639,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-05-01 02:00:00,NaN,33.803576,2.803576,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-05-01 03:00:00,NaN,30.046507,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-05-01 04:00:00,NaN,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,2023-06-15 19:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1101,2023-06-15 20:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1102,2023-06-15 21:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1103,2023-06-15 22:00:00,NaN,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
DF = df.copy()
df = df.dropna()
df.reset_index(inplace=True, drop = True)
df

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,2023-05-01 05:00:00,2.41,27.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-05-01 06:00:00,3.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-05-01 07:00:00,3.5,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-05-01 08:00:00,4.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-05-01 09:00:00,4.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2023-06-15 17:00:00,4.2,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,2023-06-15 18:00:00,4.2,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
865,2023-06-15 19:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
866,2023-06-15 20:00:00,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
hours = DF['Time'].dt.hour
df_12am_to_8am = DF[(hours >= 0) & (hours <= 8)]
df_12am_to_8am = df_12am_to_8am.sort_values('Time')
df_12am_to_8am.reset_index(inplace=True, drop = True)
df_12am_to_8am

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,2023-05-01 00:00:00,NaN,27.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-05-01 01:00:00,NaN,33.394639,2.394639,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-05-01 02:00:00,NaN,33.803576,2.803576,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-05-01 03:00:00,NaN,30.046507,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-05-01 04:00:00,NaN,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,2023-06-15 04:00:00,NaN,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
411,2023-06-15 05:00:00,3.84,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
412,2023-06-15 06:00:00,3.9,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413,2023-06-15 07:00:00,3.9,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
'''
power_change = []
for i in range(1, len(df)):
    if (pd.notna(DF['MW'].iloc[i]) and pd.notna(DF['MW'].iloc[i+1]) and (DF['Time'].iloc[i].hour - DF['Time'].iloc[i-1].hour == 1)):
        power_change.append(abs(DF['MW'].iloc[i] - DF['MW'].iloc[i-1]))

print(power_change)
indices = list(range(len(power_change)))

plt.ion()
plt.figure(figsize=(12, 6))
plt.plot(indices, power_change, marker='o', linestyle='-', color = 'y')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')
plt.title('Power Consumption Changes Over Time (e1)')
plt.grid(True)
plt.show()
'''
power_change = []
for i in range(1, len(df_12am_to_8am)):
    if (df_12am_to_8am['Time'].iloc[i].hour - df_12am_to_8am['Time'].iloc[i-1].hour == 1):
        power_change.append(abs(df_12am_to_8am['MW'].iloc[i] - df_12am_to_8am['MW'].iloc[i-1]))
indices = list(range(len(power_change)))

plt.figure(figsize=(12, 6))
plt.plot(indices, power_change, marker='o', linestyle='-', color ='b')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')
plt.title('Power Consumption Changes Over Time between 12am to 8am')
plt.grid(True)
plt.show()

"\npower_change = []\nfor i in range(1, len(df)):\n    if (pd.notna(DF['MW'].iloc[i]) and pd.notna(DF['MW'].iloc[i+1]) and (DF['Time'].iloc[i].hour - DF['Time'].iloc[i-1].hour == 1)):\n        power_change.append(abs(DF['MW'].iloc[i] - DF['MW'].iloc[i-1]))\n\nprint(power_change)\nindices = list(range(len(power_change)))\n\nplt.ion()\nplt.figure(figsize=(12, 6))\nplt.plot(indices, power_change, marker='o', linestyle='-', color = 'y')\nplt.xlabel('Time')\nplt.ylabel('Power Consumption Changes')\nplt.title('Power Consumption Changes Over Time (e1)')\nplt.grid(True)\nplt.show()\n"

<Figure size 1200x600 with 0 Axes>

Text(0.5, 0, 'Time')

Text(0, 0.5, 'Power Consumption Changes')

Text(0.5, 1.0, 'Power Consumption Changes Over Time between 12am to 8am')

In [13]:
plt.figure(figsize=(12, 6))
plt.plot(DF['Time'], DF['MW'], marker='o', linestyle='-', color ='b')
plt.xlabel('Time')
plt.ylabel('Power Consumption')
plt.title('Power Consumption Changes Over Time')
plt.grid(True)
plt.show()

<Figure size 1200x600 with 0 Axes>

Text(0.5, 0, 'Time')

Text(0, 0.5, 'Power Consumption')

Text(0.5, 1.0, 'Power Consumption Changes Over Time for August')

In [14]:
'''
power_change = []
for i in range(1, len(df)-1):
    if (pd.notna(DF['MW'].iloc[i]) and pd.notna(DF['MW'].iloc[i+1]) and pd.notna(DF['MW'].iloc[i-1]) and (DF['Time'].iloc[i].hour - DF['Time'].iloc[i-1].hour == 1) and (DF['Time'].iloc[i+1].hour - DF['Time'].iloc[i].hour == 1)):
        power_change.append(abs((DF['MW'].iloc[i+1] + DF['MW'].iloc[i-1]) - 2* DF['MW'].iloc[i]))


print(power_change)
indices = list(range(len(power_change)))

plt.ion()
plt.figure(figsize=(12, 6))
plt.plot(indices, power_change, marker='o', linestyle='-', color = 'g')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')
plt.title('Power Consumption Changes Over Time(e2)')
plt.grid(True)
plt.show()
'''

"\npower_change = []\nfor i in range(1, len(df)-1):\n    if (pd.notna(DF['MW'].iloc[i]) and pd.notna(DF['MW'].iloc[i+1]) and pd.notna(DF['MW'].iloc[i-1]) and (DF['Time'].iloc[i].hour - DF['Time'].iloc[i-1].hour == 1) and (DF['Time'].iloc[i+1].hour - DF['Time'].iloc[i].hour == 1)):\n        power_change.append(abs((DF['MW'].iloc[i+1] + DF['MW'].iloc[i-1]) - 2* DF['MW'].iloc[i]))\n\n\nprint(power_change)\nindices = list(range(len(power_change)))\n\nplt.ion()\nplt.figure(figsize=(12, 6))\nplt.plot(indices, power_change, marker='o', linestyle='-', color = 'g')\nplt.xlabel('Time')\nplt.ylabel('Power Consumption Changes')\nplt.title('Power Consumption Changes Over Time(e2)')\nplt.grid(True)\nplt.show()\n"

Try doing temperature and behavior part together

In [15]:
#tried to do the temperature and behavior part together
start_time = pd.Timestamp('2022-08-01 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')

training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)
training_df = df[training_mask]
d = training_df.loc[:,'MW']
phi_temperature = training_df.loc[:,'CDH']
phi_behavior = training_df.loc[:,'omega1':'omega168']
opti = cd.Opti()
temperature_theta = opti.variable()
behavior_theta = opti.variable(168)
total_sum =0
for i in range (0,len(d)):
    phi_behavior_i = (cd.MX(phi_behavior.iloc[i].values))
    phi_behavior_i =  cd.vertcat(phi_behavior_i)
    residual = (d.iloc[i] - ((phi_temperature.iloc[i] * temperature_theta) + cd.dot(phi_behavior_i, behavior_theta)))**2
    total_sum += residual

opti.subject_to(temperature_theta >= 0)
e1 = 3.0
e2 = 3.0

for i in range(0, 167):
    opti.subject_to((behavior_theta[i + 1] - behavior_theta[i]) <= e1)
    opti.subject_to((behavior_theta[i + 1] - behavior_theta[i]) >= -e1)
    opti.subject_to(behavior_theta[i] >= 0)
    opti.subject_to(behavior_theta[i] <= 7)

for i in range(1, 167):
    opti.subject_to((behavior_theta[i + 1] + behavior_theta[i - 1] - 2 * behavior_theta[i]) <= e2)
    opti.subject_to((behavior_theta[i + 1] + behavior_theta[i - 1] - 2 * behavior_theta[i]) >= -e2)

for i in range(7,8):
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_weeks = num_hours / 168.0
    weekly_behavior_energy_use = 0
    temperature_energy_use = 0
    for j in range(len(training_df)):
        temperature_energy_use += temperature_theta * training_df.iloc[j]['CDH']

    for j in range(0, 168):
        weekly_behavior_energy_use += behavior_theta[j]
    weekly_energy_use = weekly_behavior_energy_use +  temperature_energy_use
    opti.subject_to(weekly_energy_use < 1.5 * bill_df.iloc[i]['MW'] / num_weeks)
    opti.subject_to(weekly_energy_use > 0.5 * bill_df.iloc[i]['MW'] / num_weeks)

# Solve the optimization problem
opti.minimize(total_sum)
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()

optimal_temperature_theta = sol.value(temperature_theta)
optimal_theta_values = sol.value(behavior_theta)
print("Optimal value of temperature_theta:", optimal_temperature_theta)
print("Optimal values of theta:", optimal_theta_values)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2335
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      169
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

Temperature Dependent Part

In [16]:
'''
#temperature dependency part
start_time = pd.Timestamp('2022-08-01 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)
training_df = df[training_mask]

d = training_df.loc[:,'MW']
phi = training_df.loc[:,'CDH']
opti = cd.Opti()
theta = opti.variable()
c = opti.variable()
sum = 0
residuals = []

for i in range (0,len(d)):
    residual = (d.iloc[i] - (phi.iloc[i] * theta + c))**2
    sum += residual
    residuals.append(residual)

#objective = sqrt(sum([r**2 for r in residuals]))
objective = sqrt(sum)
opti.minimize(objective)
opti.subject_to(theta >= 0)
# Solve the optimization problem
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()

# Extract and print the optimal solution for theta
temp_theta = sol.value(theta)
print("Optimal value of theta:", temp_theta)
optimal_c = sol.value(c)
print("Optimal value of constant:", optimal_c)
'''

'\n#temperature dependency part\nstart_time = pd.Timestamp(\'2022-08-01 00:00:00\')\nend_time = pd.Timestamp(\'2022-08-28 23:00:00\')\ntraining_mask = (df[\'Time\'] >= start_time) & (df[\'Time\'] <= end_time) & (df[\'CDH\'] > 0)\ntraining_df = df[training_mask]\n\nd = training_df.loc[:,\'MW\']\nphi = training_df.loc[:,\'CDH\']\nopti = cd.Opti()\ntheta = opti.variable()\nc = opti.variable()\nsum = 0\nresiduals = []\n\nfor i in range (0,len(d)):\n    residual = (d.iloc[i] - (phi.iloc[i] * theta + c))**2\n    sum += residual\n    residuals.append(residual)\n\n#objective = sqrt(sum([r**2 for r in residuals]))\nobjective = sqrt(sum)\nopti.minimize(objective)\nopti.subject_to(theta >= 0)\n# Solve the optimization problem\nsolver_opts = {\'print_time\': 0}\nopti.solver(\'ipopt\', solver_opts)\nsol = opti.solve()\n\n# Extract and print the optimal solution for theta\ntemp_theta = sol.value(theta)\nprint("Optimal value of theta:", temp_theta)\noptimal_c = sol.value(c)\nprint("Optimal value of c

Behavior Dependent Part

In [17]:
'''
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)
behavior_df = df[training_mask]
#i think sir asked me to try with e1,e2 values as half of what I put initially which was around 2
e1 = 3.0
e2 = 3.0
#behavior dependent part
d = behavior_df['MW'] - temp_theta * behavior_df['CDH']
phi = behavior_df.loc[:,'omega1':'omega168']
opti = cd.Opti()
theta = opti.variable(168)
residuals = []
sum = 0
for i in range(len(d)):
    phi_i = (cd.MX(phi.iloc[i].values))
    phi_i =  cd.vertcat(phi_i)
    residual = (d.iloc[i] - cd.dot(phi_i, theta))**2
    sum += residual
    residuals.append(residual)

objective = sum
opti.minimize(objective)

for i in range(0, 167):
    opti.subject_to(cd.fabs(theta[i + 1] - theta[i]) <= e1)
    opti.subject_to((theta[i + 1] - theta[i]) <= e1)
    opti.subject_to((theta[i + 1] - theta[i]) >= -e1)
    opti.subject_to(theta[i] >= 0)
    opti.subject_to(theta[i] <= 7)


for i in range(1, 167):
    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) <= e2)
    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) >= -e2)


for i in range(7,8):
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_complete_weeks = num_hours // 168
    remaining_hours = (num_days*24)%168
    week_sum = 0
    for k in range(0,168):
        week_sum+= theta[k]
    sum = week_sum * num_complete_weeks
    n = first_day * 24
    for j in range(n, n+remaining_hours+1):
        sum+=theta[j%168]

    opti.subject_to(sum < bill_df.iloc[i]['MW'] + 0.01)

    opti.subject_to(sum > bill_df.iloc[i]['MW'] - 0.01)

# Solve the optimization problem
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()
time_theta = sol.value(theta)
print("Optimal value of theta:", time_theta)
'''

'\ntraining_mask = (df[\'Time\'] >= start_time) & (df[\'Time\'] <= end_time)\nbehavior_df = df[training_mask]\n#i think sir asked me to try with e1,e2 values as half of what I put initially which was around 2\ne1 = 3.0\ne2 = 3.0\n#behavior dependent part\nd = behavior_df[\'MW\'] - temp_theta * behavior_df[\'CDH\']\nphi = behavior_df.loc[:,\'omega1\':\'omega168\']\nopti = cd.Opti()\ntheta = opti.variable(168)\nresiduals = []\nsum = 0\nfor i in range(len(d)):\n    phi_i = (cd.MX(phi.iloc[i].values))\n    phi_i =  cd.vertcat(phi_i)\n    residual = (d.iloc[i] - cd.dot(phi_i, theta))**2\n    sum += residual\n    residuals.append(residual)\n\nobjective = sum\nopti.minimize(objective)\n\nfor i in range(0, 167):\n    opti.subject_to(cd.fabs(theta[i + 1] - theta[i]) <= e1)\n    opti.subject_to((theta[i + 1] - theta[i]) <= e1)\n    opti.subject_to((theta[i + 1] - theta[i]) >= -e1)\n    opti.subject_to(theta[i] >= 0)\n    opti.subject_to(theta[i] <= 7)\n\n\nfor i in range(1, 167):\n    opti.subje

In [18]:

week_start_time = pd.Timestamp('2022-08-01 00:00:00')
week_end_time = pd.Timestamp('2022-08-07 23:00:00')
week_training_mask = (DF['Time'] >= week_start_time) & (DF['Time'] <= week_end_time)
behavior_df = DF[week_training_mask]
d = behavior_df['MW']
temperature_dependent_predictions = behavior_df['CDH'] * optimal_temperature_theta

fig,(ax1) = plt.subplots(nrows=1,figsize=(10,9))
_=ax1.plot(behavior_df['Time'],d.head(168),label='meas')
_=ax1.plot(behavior_df['Time'],optimal_theta_values,label='behvaior theta:')
_=ax1.plot(behavior_df['Time'],temperature_dependent_predictions[:168],label='temperature part')
ax1.set_title('measured vs predicted data with behavior model and temperature models over first week')
_=ax1.legend()

fig,ax = plt.subplots(figsize = (10,6))
ax.plot(np.arange(0,168), optimal_theta_values,'bo-')


ValueError: x and y must have same first dimension, but have shapes (0,) and (168,)

In [ ]:
#plotting behavior model performance with DF(contains null values)
week_start_time = pd.Timestamp('2022-08-01 00:00:00')
week_end_time = pd.Timestamp('2022-08-28 23:00:00')
DF_training_mask = (DF['Time'] >= week_start_time) & (DF['Time'] <= week_end_time)
behavior_df = DF[DF_training_mask]
d = behavior_df['MW'] - (optimal_temperature_theta * behavior_df['CDH'])

fig, (ax1) = plt.subplots(nrows=1, figsize=(10, 9))
pattern_length = len(d) % len(optimal_theta_values)
extended_time_theta = np.tile(optimal_theta_values, len(d) // len(optimal_theta_values))
extended_time_theta = np.concatenate((extended_time_theta, optimal_theta_values[:pattern_length]))
# Plot all values of d against extended_time_theta
_ = ax1.plot(behavior_df['Time'], d, label='meas', marker = 'o')
_ = ax1.plot(behavior_df['Time'], extended_time_theta, label='pred:', marker = 'x')
ax1.set_title('measured vs predicted data for behavior model (contains null values)')
_ = ax1.legend()
plt.show()


In [ ]:
#final model with df(no null values)
d = training_df['MW']
c = np.concatenate((np.array([optimal_temperature_theta]),optimal_theta_values),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
full_model = LinearRegression(fit_intercept=False)
full_model.fit(phi, d)
full_model.coef_ = c
full_model.intercept_ = 0
yhat = full_model.predict(phi.values)
full_modelScore = full_model.score(phi,d)

print ('score for constructed full model on full data: ', full_modelScore)

fig,(ax1) = plt.subplots(nrows=1,figsize=(10,9))
_=ax1.plot(training_df['Time'],d,label='meas', marker = 'o')
_=ax1.plot(training_df['Time'],yhat,label='pred:', marker = 'x')
ax1.set_title('measured vs predicted data (full constructed model)')
_=ax1.legend()
